## Import Dependencies

In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pprint

In [ ]:
executable_path = {'executable_path': '/Users/vlari/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
help(pprint)

## NASA Mars News

Scrape the NASA Mars News Site ("https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest") and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

#### Example:
`news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"`

``news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."``

In [ ]:
news_Mars_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(news_Mars_url)

In [ ]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [ ]:
soup

In [ ]:
news_title = soup.find("div", class_="content_title").get_text()
news_p = soup.find("div", class_="article_teaser_body").get_text()

print(news_title)
print(news_p)

## JPL Mars Space Images - Featured Image

Visit the url for JPL Featured Space Image "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars".

Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.

Make sure to find the image url to the full size .jpg image.

Make sure to save a complete url string for this image.

In [ ]:
img_Mars_BaseUrl = "https://www.jpl.nasa.gov"
img_Mars_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(img_Mars_url)

In [ ]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [ ]:
import re

style = soup.find('article')['style']
urls = re.findall('url\((.*?)\)', style)

In [ ]:
img_EndUrl = urls[0][1:-1]
img_EndUrl

In [ ]:
featured_image_url = img_Mars_BaseUrl + img_EndUrl

In [ ]:
# Use the requests library to download and save the image from the `img_url` above
import requests
import shutil
response = requests.get(featured_image_url, stream=True)
with open('./static/images/mars_feat_img.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

In [ ]:
help(soup.find())

## Mars Weather

Visit the Mars Weather twitter account ("https://twitter.com/marswxreport?lang=en") and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

In [ ]:
twitWeather_Mars_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(twitWeather_Mars_url)

In [ ]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [ ]:
mars_weather_sol = soup.find(string = re.compile("Sol"))
print(mars_weather_sol)

In [ ]:
# This gets the latest tweet, but does not isolate weather only tweets
# mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").get_text()

# print(mars_weather)

## Mars Facts

Visit the Mars Facts webpage ("https://space-facts.com/mars/") and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

Use Pandas to convert the data to a HTML table string.

In [ ]:
facts_Mars_url = "https://space-facts.com/mars/"
browser.visit(facts_Mars_url)

In [ ]:
mars_table = pd.read_html(facts_Mars_url)
mars_table[0]

In [ ]:
Property = mars_table[0][0]
new_col = [item[:-1] for item in Property]   
new_col

In [ ]:
mars_table[0][1]

In [ ]:
mars_factDF = pd.DataFrame(data=mars_table[0][1])
mars_factDF

In [ ]:
pd.DataFrame(data=mars_table[0][1])

In [ ]:
mars_factDF.columns = ["Value"]
mars_factDF["Property"] = new_col

In [ ]:
mars_factDF

In [ ]:
mars_factDF.set_index("Property", inplace=True)

In [ ]:
mars_factDF

In [ ]:
mars_factDF.to_html('./templates/marsFactDF.html')

## Mars Hemispheres

Visit the USGS Astrogeology site ("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars") to obtain high resolution images for each of Mar's hemispheres.

You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

``hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]``

In [ ]:
hemis_Mars_BaseUrl = "https://astrogeology.usgs.gov"
hemis_Mars_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemis_Mars_url)

In [ ]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [ ]:
mars_hemis = soup.find("div", class_="collapsible results").find_all("div", class_="item")
titles = [item.find("h3").get_text() for item in mars_hemis]

In [ ]:
urls = [hemis_Mars_BaseUrl + item.find("a", class_="itemLink product-item").get("href") for item in mars_hemis]

In [ ]:
hemisphere_image_urls

In [ ]:
def listTOdict(keys, lists):
    if len(keys) != len(lists):
        print("Parameters must be equal in length")
        return []
    rows = len(lists[0])
    cols = len(keys)
    listed = []
    for r in range(rows):
        entry = {}    
        for c in range(cols):
            entry[keys[c]] = lists[c][r]
        listed.append(entry)
    return listed

In [ ]:
marsKeys = ["title", "url"]
marsLists = [titles, urls]

hemisphere_image_urls = listTOdict(marsKeys, marsLists)